In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from config import user_name,password
import MySQLdb
from sqlalchemy import create_engine

In [2]:
#find all 30 Dow Jone Industrial stock tickers
url = "https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('a', class_="C($c-fuji-blue-1-b) Cur(p) Td(n) Fw(500)")
tickers1=[]

for x in results:
    tickers1.append(x.text)



In [3]:
results1=soup.find_all('td', class_="Py(10px) Ta(start) Pend(10px)")
company_name=[]
for x in range(1,61,2):
    company_name.append(results1[x].text)
company_name

['International Business Machines Corporation',
 'Microsoft Corporation',
 'Exxon Mobil Corporation',
 '3M Company',
 'Chevron Corporation',
 'The Procter & Gamble Company',
 'Dow Inc.',
 'The Goldman Sachs Group, Inc.',
 'The Home Depot, Inc.',
 'Cisco Systems, Inc.',
 'Intel Corporation',
 'Pfizer Inc.',
 'Walgreens Boots Alliance, Inc.',
 'Visa Inc.',
 'Walmart Inc.',
 'United Technologies Corporation',
 "McDonald's Corporation",
 'JPMorgan Chase & Co.',
 'NIKE, Inc.',
 'Verizon Communications Inc.',
 'The Coca-Cola Company',
 'The Walt Disney Company',
 'Johnson & Johnson',
 'Apple Inc.',
 'UnitedHealth Group Incorporated',
 'Merck & Co., Inc.',
 'The Travelers Companies, Inc.',
 'Caterpillar Inc.',
 'American Express Company',
 'The Boeing Company']

In [4]:
Company_Name_Table=pd.DataFrame()
Company_Name_Table["Ticker"]=tickers1
Company_Name_Table["Company_Name"]=company_name
Company_Name_Table=Company_Name_Table.sort_values(by="Ticker")
Company_Name_Table=Company_Name_Table.reset_index(drop=True)
Company_Name_Table.insert(0,"Ticker_ID",Company_Name_Table.index+1)
Company_Name_Table.tail()

,Ticker_ID,Ticker,Company_Name
25,26,V,Visa Inc.
26,27,VZ,Verizon Communications Inc.
27,28,WBA,"Walgreens Boots Alliance, Inc."
28,29,WMT,Walmart Inc.
29,30,XOM,Exxon Mobil Corporation


In [5]:
tickers=tickers1.copy()
tickers.sort()


In [6]:
# URL of page to be scraped
url = "https://finance.yahoo.com/quote/WMT/key-statistics?p=WMT"
df=pd.DataFrame()
df1=pd.DataFrame()
df2=pd.DataFrame()
df3=pd.DataFrame()
df4=pd.DataFrame()
df5=pd.DataFrame()
df6=pd.DataFrame()
df7=pd.DataFrame()
df8=pd.DataFrame()
tables = pd.read_html(url)
df=pd.DataFrame(tables[0][0])

In [7]:
for x in tickers:
    url= f"https://finance.yahoo.com/quote/{x}/key-statistics?p={x}"
    k=pd.read_html(url)
    df[x]=k[0][1]
    df1[x]=k[7][1]
    df2[x]=k[8][1]
    df3[x]=k[9][1]
    df4[x]=k[2][1]
    df5[x]=k[4][1]
    df6[x]=k[5][1]
    df7[x]=k[6][1]
    df8[x]=k[3][1]

In [8]:
dfcopy=df.T.copy()
dfcopy.columns = dfcopy.iloc[0]
dfcopy=dfcopy[1:]
dfcopy.reset_index(level=0,inplace=True)
dfcopy.insert(0, "Ticker_ID", dfcopy.index+1)
Valuation_Measures_Table=dfcopy.rename(columns={"index":"Ticker"})
Valuation_Measures_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Valuation_Measures_Table=Valuation_Measures_Table.drop(["Ticker"], axis=1)
Valuation_Measures_Table.head()

,Ticker_ID,Company_Name,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6
0,1,Apple Inc.,932.13B,964.67B,17.04,16.07,1.50,3.61,8.82,3.73,12.47
1,2,American Express Company,104.23B,137.85B,15.44,13.92,1.48,2.72,4.52,3.59,NaN
2,3,The Boeing Company,212.31B,220.5B,21.60,16.48,2.10,2.11,1699.82,2.19,16.39
3,4,Caterpillar Inc.,77.91B,108.11B,12.67,10.61,0.72,1.41,5.05,1.95,9.71
4,5,"Cisco Systems, Inc.",245.54B,234.61B,19.90,16.82,2.16,4.78,6.72,4.57,15.16


In [9]:
Stock_Price_Hist_Table = df1.T.copy() 
Stock_Price_Hist_Table.columns=tables[7][0]
Stock_Price_Hist_Table.reset_index(level=0,inplace=True)
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns = lambda x : str(x)[:-1])
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={"Beta (3Y Monthly":"Beta (3Y Monthly)"})
Stock_Price_Hist_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={"inde":"Ticker"})
Stock_Price_Hist_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Stock_Price_Hist_Table=Stock_Price_Hist_Table.drop(["Ticker"], axis=1)
Stock_Price_Hist_Table.columns
Stock_Price_Hist_Table=Stock_Price_Hist_Table.rename(columns={'52-Week Change ':'52 Week Change',\
    'S&P500 52-Week Change ':'S&P500 52 Week Change','50-Day Moving Average ':'50 Day Moving Average',\
        '200-Day Moving Average ':'200 Day Moving Average','52 Week High ':'52 Week High',\
        '52 Week Low ':'52 Week Low'})
Stock_Price_Hist_Table.head()


,Ticker_ID,Company_Name,Revenue (ttm,Revenue Per Share (ttm,Quarterly Revenue Growth (yoy,Gross Profit (ttm,EBITD,Net Income Avi to Common (ttm,Diluted EPS (ttm,Quarterly Earnings Growth (yoy
0,1,Apple Inc.,258.49B,54.15,-5.10%,101.84B,77.34B,57.17B,11.89,-16.40%
1,2,American Express Company,38.38B,45.42,8.50%,25.51B,NaN,6.84B,8.09,8.50%
2,3,The Boeing Company,100.66B,175.33,-2.00%,19.64B,13.45B,10.13B,17.47,-13.20%
3,4,Caterpillar Inc.,55.33B,94.58,4.70%,14.9B,11.13B,6.36B,10.75,13.00%
4,5,"Cisco Systems, Inc.",51.32B,11.36,4.00%,30.68B,15.47B,13.22B,2.88,13.10%


In [10]:
Share_Statistics_Table=df2.T.copy()
Share_Statistics_Table.columns=tables[8][0]
Share_Statistics_Table.reset_index(level=0,inplace=True)
Share_Statistics_Table=Share_Statistics_Table.rename(columns = lambda x : str(x)[:-1])
Share_Statistics_Table=Share_Statistics_Table.rename(columns=lambda x: x.strip())
Share_Statistics_Table=Share_Statistics_Table.rename(columns={"Floa":"Float"})
Share_Statistics_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Share_Statistics_Table=Share_Statistics_Table.rename(columns={"inde":"Ticker"})
Share_Statistics_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Share_Statistics_Table=Share_Statistics_Table.drop(["Ticker"], axis=1)
Share_Statistics_Table.head()


,Ticker_ID,Company_Name,Total Cash (mrq,Total Cash Per Share (mrq,Total Debt (mrq,Total Debt/Equity (mrq,Current Ratio (mrq,Book Value Per Share (mrq
0,1,Apple Inc.,80.09B,17.41,112.63B,106.39,1.32,22.98
1,2,American Express Company,27B,32.45,61B,265.22,2.18,27.64
2,3,The Boeing Company,7.73B,13.74,15.81B,6813.79,1.07,0.22
3,4,Caterpillar Inc.,6.36B,11.12,36.54B,236.08,1.43,27.00
4,5,"Cisco Systems, Inc.",34.64B,8.09,23.71B,64.42,1.70,8.53


In [11]:
Dividends_Splits_Table=df3.T.copy()
Dividends_Splits_Table.columns=tables[9][0]
Dividends_Splits_Table.reset_index(level=0,inplace=True)
Dividends_Splits_Table=Dividends_Splits_Table.rename(columns = lambda x : str(x)[:-2])
Dividends_Splits_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Dividends_Splits_Table=Dividends_Splits_Table.rename(columns={"ind":"Ticker"})
Dividends_Splits_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Dividends_Splits_Table=Dividends_Splits_Table.drop(["Ticker"], axis=1)
Dividends_Splits_Table.head()


,Ticker_ID,Company_Name,Operating Cash Flow (tt,Levered Free Cash Flow (tt
0,1,Apple Inc.,71.86B,45.63B
1,2,American Express Company,NaN,NaN
2,3,The Boeing Company,14.97B,8.45B
3,4,Caterpillar Inc.,6.74B,2.56B
4,5,"Cisco Systems, Inc.",15.99B,9.71B


In [12]:
Profitability_Table=df4.T.copy()
Profitability_Table.columns=tables[2][0]
Profitability_Table.reset_index(level=0,inplace=True)
Profitability_Table=Profitability_Table.rename(columns={"index":"Ticker"})
Profitability_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Profitability_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Profitability_Table=Profitability_Table.drop(["Ticker"], axis=1)
Profitability_Table.head()


,Ticker_ID,Company_Name,Avg Vol (3 month) 3,Avg Vol (10 day) 3,Shares Outstanding 5,Float,% Held by Insiders 1,% Held by Institutions 1,"Shares Short (Jun 28, 2019) 4","Short Ratio (Jun 28, 2019) 4","Short % of Float (Jun 28, 2019) 4","Short % of Shares Outstanding (Jun 28, 2019) 4","Shares Short (prior month May 31, 2019) 4"
0,1,Apple Inc.,26.71M,17.89M,4.6B,4.35B,0.07%,60.95%,43.45M,1.68,0.92%,0.94%,51.26M
1,2,American Express Company,3.11M,3.32M,835.08M,678.82M,0.29%,86.33%,6.18M,2.21,0.90%,0.74%,7.11M
2,3,The Boeing Company,4.5M,4.94M,582.58M,562.23M,0.07%,70.56%,5.63M,1.44,1.00%,1.00%,5.62M
3,4,Caterpillar Inc.,4.36M,4M,571.88M,570.87M,0.17%,68.58%,7.36M,1.8,1.29%,1.29%,10.65M
4,5,"Cisco Systems, Inc.",21.91M,11.92M,4.28B,4.28B,0.07%,77.33%,47.54M,1.84,1.03%,1.11%,43.67M


In [13]:
Management_Effectiveness_Table=df8.T.copy()
Management_Effectiveness_Table.columns=tables[3][0]
Management_Effectiveness_Table.reset_index(level=0,inplace=True)
Management_Effectiveness_Table=Management_Effectiveness_Table.rename(columns={"index":"Ticker"})
Management_Effectiveness_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Management_Effectiveness_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Management_Effectiveness_Table=Management_Effectiveness_Table.drop(["Ticker"], axis=1)
Management_Effectiveness_Table.head()


,Ticker_ID,Company_Name,Forward Annual Dividend Rate 4,Forward Annual Dividend Yield 4,Trailing Annual Dividend Rate 3,Trailing Annual Dividend Yield 3,5 Year Average Dividend Yield 4,Payout Ratio 4,Dividend Date 3,Ex-Dividend Date 4,Last Split Factor (new per old) 2,Last Split Date 3
0,1,Apple Inc.,3.08,1.50%,2.92,1.42%,1.66,24.46%,"May 16, 2019","May 10, 2019",1/7,"Jun 9, 2014"
1,2,American Express Company,1.56,1.21%,1.56,1.21%,1.46,19.36%,"Aug 9, 2019","Jul 3, 2019",8753/10000,"Oct 3, 2005"
2,3,The Boeing Company,8.22,2.28%,7.18,1.99%,2.31,41.17%,"Sep 6, 2019","Aug 8, 2019",1/2,"Jun 9, 1997"
3,4,Caterpillar Inc.,4.12,3.05%,3.44,2.57%,3.02,31.20%,"Aug 20, 2019","Jul 19, 2019",1/2,"Jul 14, 2005"
4,5,"Cisco Systems, Inc.",1.4,2.42%,1.34,2.32%,2.94,46.69%,"Jul 24, 2019","Jul 3, 2019",1/2,"Mar 23, 2000"


In [14]:
tables[2][0]

0                                Avg Vol (3 month) 3
1                                 Avg Vol (10 day) 3
2                               Shares Outstanding 5
3                                              Float
4                               % Held by Insiders 1
5                           % Held by Institutions 1
6                      Shares Short (Jun 28, 2019) 4
7                       Short Ratio (Jun 28, 2019) 4
8                  Short % of Float (Jun 28, 2019) 4
9     Short % of Shares Outstanding (Jun 28, 2019) 4
10         Shares Short (prior month May 31, 2019) 4
Name: 0, dtype: object

In [15]:
tables[4]

,0,1
0,Fiscal Year Ends,"Jan 31, 2019"
1,Most Recent Quarter (mrq),"Apr 30, 2019"


In [16]:
Income_Statement_Table=df5.T.copy()
Income_Statement_Table.columns=tables[4][0]
Income_Statement_Table.reset_index(level=0,inplace=True)
Income_Statement_Table=Income_Statement_Table.rename(columns={"index":"Ticker"})
Income_Statement_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Income_Statement_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Income_Statement_Table=Income_Statement_Table.drop(["Ticker"], axis=1)
Income_Statement_Table.head()

,Ticker_ID,Company_Name,Fiscal Year Ends,Most Recent Quarter (mrq)
0,1,Apple Inc.,"Sep 29, 2018","Mar 30, 2019"
1,2,American Express Company,"Dec 31, 2018","Jun 30, 2019"
2,3,The Boeing Company,"Dec 31, 2018","Mar 31, 2019"
3,4,Caterpillar Inc.,"Dec 31, 2018","Mar 31, 2019"
4,5,"Cisco Systems, Inc.","Jul 28, 2018","Apr 27, 2019"


In [17]:
Balance_Sheet_Table=df6.T.copy()
Balance_Sheet_Table.columns=tables[5][0]
Balance_Sheet_Table.reset_index(level=0,inplace=True)
Balance_Sheet_Table=Balance_Sheet_Table.rename(columns={"index":"Ticker"})
Balance_Sheet_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Balance_Sheet_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Balance_Sheet_Table=Balance_Sheet_Table.drop(["Ticker"], axis=1)
Balance_Sheet_Table.head()


,Ticker_ID,Company_Name,Profit Margin,Operating Margin (ttm)
0,1,Apple Inc.,22.12%,25.34%
1,2,American Express Company,18.17%,21.28%
2,3,The Boeing Company,10.06%,11.24%
3,4,Caterpillar Inc.,11.50%,15.20%
4,5,"Cisco Systems, Inc.",25.76%,26.80%


In [18]:
Cash_Flow_Table=df7.T.copy()
Cash_Flow_Table.columns=tables[6][0]
Cash_Flow_Table.reset_index(level=0,inplace=True)
Cash_Flow_Table=Cash_Flow_Table.rename(columns={"index":"Ticker"})
Cash_Flow_Table.insert(0, "Ticker_ID", dfcopy.index+1)
Cash_Flow_Table.insert(1,"Company_Name", Company_Name_Table["Company_Name"].copy())
Cash_Flow_Table=Cash_Flow_Table.drop(["Ticker"], axis=1)
Cash_Flow_Table.head()


,Ticker_ID,Company_Name,Return on Assets (ttm),Return on Equity (ttm)
0,1,Apple Inc.,11.54%,49.13%
1,2,American Express Company,3.64%,31.78%
2,3,The Boeing Company,6.05%,"1,324.44%"
3,4,Caterpillar Inc.,6.71%,41.40%
4,5,"Cisco Systems, Inc.",8.14%,31.67%


In [19]:
url="https://tradingeconomics.com/united-states/stock-market"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
results2 = soup.find_all('td', class_="hidden-xs")
othertickers=[]
daily_change=[]
daily_percent_change=[]
yearly_percent_change=[]
price=[]
for x in results2:
        if x.b:
           othertickers.append(x.b.text)
othertickers=othertickers[:30]

In [20]:
results3=soup.find_all('td', id="nch")
for x in results3:
    daily_change.append(x.text.strip())
daily_change=daily_change[:30]                       
results4=soup.find_all('td', id="pch")
for x in results4:
    daily_percent_change.append(x.text.strip())
daily_percent_change=daily_percent_change[:30]
results5=soup.find_all('td')
for x in range(6,276,9):
    yearly_percent_change.append(results5[x].text)
yearly_percent_change=yearly_percent_change[:30]
results6=soup.find_all('td', id="p")
for x in results6:
    price.append(x.text.strip())
price=price[:30]
General_Table=pd.DataFrame()
General_Table["Ticker"]=othertickers
General_Table["Price"]=price
General_Table["Daily_Change"]=daily_change
General_Table["Daily_Percent_Change"]=daily_percent_change
General_Table["Yearly_Percent_Change"]=yearly_percent_change
General_Table=General_Table.sort_values(by="Ticker")
General_Table=General_Table.reset_index(drop=True)
General_Table.insert(0,"Ticker_ID", General_Table.index+1)
General_Table.head()

,Ticker_ID,Ticker,Price,Daily_Change,Daily_Percent_Change,Yearly_Percent_Change
0,1,AAPL,202.59,-3.07,-1.49%,5.74%
1,2,AXP,124.82,-3.58,-2.79%,23.35%
2,3,BA,377.36,16.25,4.50%,6.82%
3,4,CAT,136.23,1.34,0.99%,-0.02%
4,5,CSCO,57.36,-0.38,-0.66%,36.54%


In [21]:
url="https://finance.yahoo.com/quote/HD/profile?p=HD"

In [22]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
newresult = soup.find_all('span', class_="Fw(600)")
newresult

[<span class="Fw(600)" data-reactid="21">Consumer Cyclical</span>,
 <span class="Fw(600)" data-reactid="25">Home Improvement Stores</span>,
 <span class="Fw(600)" data-reactid="29"><span data-reactid="30">400,000</span></span>]

In [23]:
sector=[]
industry=[]
employee_number=[]
for x in tickers:
    url=f"https://finance.yahoo.com/quote/{x}/profile?p={x}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    newresult1 = soup.find_all('span', class_="Fw(600)")
    sector.append(newresult1[0].text)
    industry.append(newresult1[1].text)
    employee_number.append(newresult1[2].text)

In [24]:
Generalinfo=pd.DataFrame()
Generalinfo["Ticker"]=tickers
Generalinfo["Industry"]=industry
Generalinfo["Sector"]=sector
Generalinfo["Numbers_of_Employee"]=employee_number
Generalinfo.insert(0,"Ticker_ID",Generalinfo.index+1)
Generalinfo.head()

,Ticker_ID,Ticker,Industry,Sector,Numbers_of_Employee
0,1,AAPL,Consumer Electronics,Technology,"100,000"
1,2,AXP,Credit Services,Financial Services,"59,000"
2,3,BA,Aerospace & Defense,Industrials,"153,000"
3,4,CAT,Farm & Construction Equipment,Industrials,"104,000"
4,5,CSCO,Communication Equipment,Technology,"74,200"


In [25]:
Industry_Table=pd.DataFrame()
Industry_Table["Industry"]=industry
Industry_Table["Sector"]=sector
Industry_Table=Industry_Table.drop_duplicates(subset='Industry', keep="last")
Industry_Table=Industry_Table.sort_values(by="Industry")
Industry_Table=Industry_Table.reset_index(drop=True)
Industry_Table.insert(0,"Industry_ID",Industry_Table.index+1)
Industry_Table
mergetable=pd.merge(Generalinfo,Industry_Table,on="Industry")
mergetable=mergetable.drop(["Sector_x","Numbers_of_Employee"],axis=1)
mergetable=mergetable.rename(columns={"Sector_y":"Sector"})
mergetable.head()

,Ticker_ID,Ticker,Industry,Industry_ID,Sector
0,1,AAPL,Consumer Electronics,7,Technology
1,2,AXP,Credit Services,8,Financial Services
2,26,V,Credit Services,8,Financial Services
3,3,BA,Aerospace & Defense,1,Industrials
4,25,UTX,Aerospace & Defense,1,Industrials


In [26]:
Sector_Table=pd.DataFrame()
Sector_Table["Sector"]=sector
Sector_Table=Sector_Table.drop_duplicates(subset='Sector', keep="last")
Sector_Table=Sector_Table.reset_index(drop=True)
Sector_Table.insert(0,"Sector_ID",Sector_Table.index+1)
Sector_Table.head()

,Sector_ID,Sector
0,1,Basic Materials
1,2,Technology
2,3,Consumer Cyclical
3,4,Healthcare
4,5,Industrials


In [27]:
mergetable1=pd.merge(mergetable,Sector_Table,on="Sector")
mergetable1=mergetable1.sort_values(by="Ticker_ID")
mergetable1=mergetable1.reset_index(drop=True)
Stock_Type_Table=mergetable1.drop(["Industry","Sector"], axis=1)
Stock_Type_Table.head()

,Ticker_ID,Ticker,Industry_ID,Sector_ID
0,1,AAPL,7,2
1,2,AXP,8,6
2,3,BA,1,5
3,4,CAT,12,5
4,5,CSCO,6,2


In [28]:
Industry_Table=mergetable1[["Industry_ID","Industry","Sector_ID"]]
Industry_Table=Industry_Table.drop_duplicates(subset="Industry",keep="last")
Industry_Table=Industry_Table.sort_values(by="Industry_ID")
Industry_Table=Industry_Table.reset_index(drop=True)
Industry_Table.head(10)

,Industry_ID,Industry,Sector_ID
0,1,Aerospace & Defense,5
1,2,Banks - Global,6
2,3,Beverages - Soft Drinks,8
3,4,Capital Markets,6
4,5,Chemicals,1
5,6,Communication Equipment,2
6,7,Consumer Electronics,2
7,8,Credit Services,6
8,9,Discount Stores,8
9,10,Diversified Industrials,5


In [29]:
#Industry_Table,Stock_Type_Table,Sector_Table,General_Table

In [30]:
#Valuation_Measures_Table,Stock_Price_Hist_Table,Dividends_Splits_Table,Share_Statistics_Table,Profitability_Table,Management_Effectiveness_Table,
#Income_Statement_Table,Balance_Sheet_Table,Cash_Flow_Table

#Company_Name_Table

In [31]:
aa=[]
kk=[]
for x in tickers:
    url=f"https://money.cnn.com/quote/profile/profile.html?symb={x}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    newresult2=soup.find_all("div", class_="wsod_companyContactInfo wsod_companyPhoneURL")
    newresult = soup.find_all('div', class_="wsod_companyContactInfo wsod_companyNameStreet")
    for x in newresult:
        zz=x.find_all('div')
        for z in zz:
            aa.append(z.text)
                
    for x in newresult2:
        ff=x.find_all('div')
        for v in ff:
            kk.append(v.text)

In [32]:
aa[2]=aa[2][:-5]
aa[5]=aa[5][:-5]
aa[8]=aa[8][:-5]
aa[14]=aa[14][:-5]
aa[17]=aa[17][:-5]
aa[20]=aa[20][:-5]
aa[29]=aa[29][:-5]
aa[32]=aa[32][:-5]
aa[35]=aa[35][:-5]
aa[41]=aa[41][:-5]
aa[44]=aa[44][:-5]
aa[50]=aa[50][:-5]
aa[56]=aa[56][:-5]
aa[62]=aa[62][:-5]
aa[65]=aa[65][:-5]
aa[68]=aa[68][:-5]
aa[77]=aa[77][:-5]
aa[86]=aa[86][:-5]
aa[89]=aa[89][:-5]


In [33]:
add=[]
for x in range(1,90,3):
    add.append(aa[x]+", "+aa[x+1][:-6]+", "+aa[x+1][-5:])
add
zipcode=[]
for x in range(1,90,3):
    zipcode.append(aa[x+1][-5:])
phone=[]
for x in range(0,90,3):
    phone.append(kk[x][2:])
website=[]
for x in range(0,90,3):
    website.append(kk[x+2])

In [34]:
rawAddress_Table=pd.DataFrame()
rawAddress_Table["Ticker"]=tickers
rawAddress_Table["Address"]=add
rawAddress_Table["Zipcode"]=zipcode
rawAddress_Table.insert(0,"Ticker_ID",rawAddress_Table.index+1)
rawAddress_Table["Website"]=website
rawAddress_Table["Phone"]=phone
Company_Contact_Talbe=rawAddress_Table
Company_Contact_Talbe["Zipcode"]=pd.to_numeric(Company_Contact_Talbe["Zipcode"])
Company_Contact_Talbe.head()

,Ticker_ID,Ticker,Address,Zipcode,Website,Phone
0,1,AAPL,"One Apple Park Way, Cupertino California, 95014",95014,www.apple.com,(408) 996-1010
1,2,AXP,"200 Vesey Street, New York New York, 10285",10285,www.americanexpress.com,(212) 640-2000
2,3,BA,"100 North Riverside Plaza, Chicago Illinois, 6...",60606,www.boeing.com,(312) 544-2000
3,4,CAT,"510 Lake Cook Road, Deerfield Illinois, 60015",60015,www.caterpillar.com,(224) 551-4000
4,5,CSCO,"170 West Tasman Drive, San Jose California, 95134",95134,www.cisco.com,(408) 526-4000


In [35]:
uszip=pd.read_csv("uszips.csv",encoding="utf-8")
newcol=['city', 'state_id', 'state_name','zip', 'lat', 'lng']
uszip=uszip[newcol]
Zip_Table=uszip
Zip_Table.tail()

,city,state_id,state_name,zip,lat,lng
33094,Hyder,AK,Alaska,99923,55.9811,-130.0405
33095,Klawock,AK,Alaska,99925,55.5577,-132.9763
33096,Metlakatla,AK,Alaska,99926,55.1262,-131.4893
33097,Point Baker,AK,Alaska,99927,56.2510,-133.3757
33098,Wrangell,AK,Alaska,99929,56.3670,-131.9329


In [36]:
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user=f"{user_name}",         # your username
                     passwd=f"{password}",  # your password
                     use_unicode=True,
                     charset="utf8"
                     
                        )        # name of the data base
cursor = db.cursor()


PROJECT_DB="ETL_STOCK_DB"
# Execute SQL select statement
cursor.execute(f"DROP DATABASE IF EXISTS {PROJECT_DB}")

cursor.execute(f"CREATE DATABASE IF NOT EXISTS {PROJECT_DB}")
cursor.close()
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user=f"{user_name}",         # your username
                     passwd=f"{password}",  # your password
                     use_unicode=True,
                     charset="utf8",
                     database=f"{PROJECT_DB}"
                        )        # name of the data base
cursor = db.cursor()



In [37]:
rds_connection_string = f"{user_name}:{password}@127.0.0.1/{PROJECT_DB}"
engine = create_engine(f'mysql://{rds_connection_string}?charset=utf8')

In [38]:
#Valuation_Measures_Table,Stock_Price_Hist_Table,Dividends_Splits_Table,Share_Statistics_Table,Profitability_Table,Management_Effectiveness_Table,
#Income_Statement_Table,Balance_Sheet_Table,Cash_Flow_Table

#Company_Name_Table

#Industry_Table,Stock_Type_Table,Sector_Table,General_Table

In [39]:
General_Table.to_sql(name='stock_summary', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `stock_summary` ADD PRIMARY KEY (`TICKER_ID`)')

Valuation_Measures_Table.to_sql(name='valuation_measures', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `valuation_measures` ADD PRIMARY KEY (`TICKER_ID`)')

Stock_Price_Hist_Table.to_sql(name='price_history', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `price_history` ADD PRIMARY KEY (`TICKER_ID`)')

Dividends_Splits_Table.to_sql(name='dividends_splits', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `dividends_splits` ADD PRIMARY KEY (`TICKER_ID`)')

Share_Statistics_Table.to_sql(name='share_stats', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `share_stats` ADD PRIMARY KEY (`TICKER_ID`)')

Profitability_Table.to_sql(name='profitability', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `profitability` ADD PRIMARY KEY (`TICKER_ID`)')

Management_Effectiveness_Table.to_sql(name='manage_effectiveness', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `manage_effectiveness` ADD PRIMARY KEY (`TICKER_ID`)')

Income_Statement_Table.to_sql(name='income_statement', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `income_statement` ADD PRIMARY KEY (`TICKER_ID`)')

Balance_Sheet_Table.to_sql(name='balance_sheet', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `balance_sheet` ADD PRIMARY KEY (`TICKER_ID`)')

Cash_Flow_Table.to_sql(name='cash_flow', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `cash_flow` ADD PRIMARY KEY (`TICKER_ID`)')


0

In [40]:
Company_Name_Table.to_sql(name='company_name', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `company_name` ADD PRIMARY KEY (`TICKER_ID`)')

Company_Contact_Talbe.to_sql(name='contact_info', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `contact_info` ADD PRIMARY KEY (`TICKER_ID`)')

Zip_Table.to_sql(name='zipcode', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `zipcode` ADD PRIMARY KEY (`zip`)')

0

In [41]:
Industry_Table.to_sql(name='industry', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `industry` ADD PRIMARY KEY (`industry_id`)')

Stock_Type_Table.to_sql(name='profile_info', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `profile_info` ADD PRIMARY KEY (`ticker_id`)')

Sector_Table.to_sql(name='sector', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `sector` ADD PRIMARY KEY (`sector_id`)')

Zip_Table.to_sql(name='zipcode', con=engine, if_exists='replace', index=False)
cursor.execute('Alter Table `zipcode` ADD PRIMARY KEY (`zip`)')

0

In [42]:
cursor.execute('ALTER TABLE `stock_summary` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `contact_info` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profile_info` ADD FOREIGN KEY (`ticker_id`) REFERENCES `company_name`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profile_info` ADD FOREIGN KEY (`industry_id`) REFERENCES `industry`(`industry_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `industry` ADD FOREIGN KEY (`sector_id`) REFERENCES `sector`(`sector_id`) ON DELETE CASCADE')


25

In [43]:
cursor.execute('ALTER TABLE `valuation_measures` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `price_history` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `dividends_splits` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `share_stats` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `profitability` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `manage_effectiveness` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `income_statement` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `balance_sheet` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')
cursor.execute('ALTER TABLE `cash_flow` ADD FOREIGN KEY (`ticker_id`) REFERENCES `stock_summary`(`ticker_id`) ON DELETE CASCADE')


30